In [72]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import sys

In [73]:
sys.path.append('/groups/icecube/cyan/Utils')
from PlotUtils import setMplParam, getColour, getHistoParam 
# getHistoParam:
# Nbins, binwidth, bins, counts, bin_centers  = 
from DB_lister import list_content, list_tables
from ExternalFunctions import nice_string_output, add_text_to_ax
setMplParam()

In [74]:
# df_lookup = pd.DataFrame([
#     ["20250518", "152548", 512, 128, 6, 4, 2, "ReLu", "scaled_dot", "Absolute", "A4500"],
#     ["20250518", "153151", 256, 128, 6, 4, 2, "ReLu", "scaled_dot", "Absolute", "A4500"],
#     ["20250518", "233656", 512, 256, 6, 4, 2, "ReLu", "scaled_dot", "Absolute", "A4500"],
#     ["20250518", "233931", 512, 256, 8, 4, 2, "ReLu", "scaled_dot", "Absolute", "A4500"],
#     ["20250518", "172719", 512, 256, 6, 1, 2, "ReLu", "scaled_dot", "Absolute", "A4500"],
#     ["20250518", "172825", 512, 256, 10, 4, 2, "ReLu", "scaled_dot", "Absolute", "A4500"],
#     ["20250518", "174841", 512, 256, 10, 8, 2, "ReLu", "scaled_dot", "Absolute", "A4500"],
#     ["20250518", "174943", 512, 256, 10, 8, 4, "ReLu", "scaled_dot", "Absolute", "A4500"],
#     ["20250518", "235356", 512, 256, 10, 8, 8, "ReLu", "scaled_dot", "Absolute", "A4500"],
#     ["20250518", "235535", 512, 256, 10, 8, 8, "GELU", "scaled_dot", "Absolute", "A4500"],
#     ["20250519", "111436", 512, 256, 10, 8, 8, "SiLU", "scaled_dot", "Absolute", "A4500"],
#     ["20250519", "111753", 512, 256, 10, 8, 8, "SiLU", "xformers",  "Absolute", "A4500"],
#     ["20250519", "112057", 512, 256, 10, 8, 8, "SiLU", "xformers",  "ROPE",     "A4500"],
#     ["20250519", "113232", 512, 256, 10, 8, 8, "SiLU", "ALiBi",     "ALiBi",    "A4500"],
#     ["20250519", "173628", 512, 256, 10, 8, 8, "SiLU", "T5",        "T5",       "A4500"],
#     ["20250519", "180656", 512, 256, 10, 8, 8, "SiLU", "scaled_dot","ROPE",     "A4500"],
#     ["20250521", "121428", 512, 256, 10, 8, 8, "GELU", "scaled_dot","ROPE",     "A4500"],
#     ["20250521", "121701", 512, 256, 10, 8, 8, "GELU", "ALiBi",     "ALiBi",     "A4500"],
#     ["20250521", "122145", 512, 256, 10, 8, 8, "GELU", "T5",        "T5",        "A4500"],
# ], columns=[
#     "date", "id", "d_model", "seq_len", "n_layers", "n_heads",
#     "n_output_layers", "out_activation", "attention", "position_encoding", "GPU"
# ])


In [92]:
df_lookup = pd.DataFrame([
    ["1", "20250518", "152548", 512, 128, 6, 4, 2, "ReLu", "ReLu", "scaled_dot", "Absolute", "A4500"],
    ["2", "20250518", "153151", 256, 128, 6, 4, 2, "ReLu", "ReLu", "scaled_dot", "Absolute", "A4500"],
    ["3", "20250518", "233656", 512, 256, 6, 4, 2, "ReLu", "ReLu", "scaled_dot", "Absolute", "A4500"],
    ["4", "20250518", "233931", 512, 256, 8, 4, 2, "ReLu", "ReLu", "scaled_dot", "Absolute", "A4500"],
    ["5", "20250518", "172719", 512, 256, 6, 1, 2, "ReLu", "ReLu", "scaled_dot", "Absolute", "A4500"],
    ["6", "20250518", "172825", 512, 256, 10, 4, 2, "ReLu", "ReLu", "scaled_dot", "Absolute", "A4500"],
    ["7", "20250518", "174841", 512, 256, 10, 8, 2, "ReLu", "ReLu", "scaled_dot", "Absolute", "A4500"],
    ["8", "20250518", "174943", 512, 256, 10, 8, 4, "ReLu", "ReLu", "scaled_dot", "Absolute", "A4500"],
    ["9", "20250518", "235356", 512, 256, 10, 8, 8, "ReLu", "ReLu", "scaled_dot", "Absolute", "A4500"],
    ["10", "20250518", "235535", 512, 256, 10, 8, 8, "GELU", "GELU", "scaled_dot", "Absolute", "A4500"],
    ["11", "20250519", "111436", 512, 256, 10, 8, 8, "SiLU", "SiLU", "scaled_dot", "Absolute", "A4500"],
    ["12", "20250519", "111753", 512, 256, 10, 8, 8, "SiLU", "SiLU", "xformers", "Absolute", "A4500"],
    ["13", "20250519", "112057", 512, 256, 10, 8, 8, "SiLU", "SiLU", "xformers", "ROPE", "A4500"],
    ["14", "20250519", "113232", 512, 256, 10, 8, 8, "SiLU", "SiLU", "AliBi", "AliBi", "A4500"],
    ["15", "20250519", "173628", 512, 256, 10, 8, 8, "SiLU", "SiLU", "T5", "T5", "A4500"],
    ["16", "20250519", "180656", 512, 256, 10, 8, 8, "SiLU", "SiLU", "scaled_dot", "ROPE", "A4500"],
    ["17", "20250521", "121428", 512, 256, 10, 8, 8, "GELU", "SiLU", "scaled_dot", "ROPE", "A4500"],
    ["18", "20250521", "121701", 512, 256, 10, 8, 8, "GELU", "SiLU", "AliBi", "AliBi", "A4500"],
    ["19", "20250521", "122145", 512, 256, 10, 8, 8, "GELU", "SiLU", "T5", "T5", "A4500"],
    ["20", "20250522", "170426", 512, 256, 10, 8, 8, "GELU", "GELU", "T5", "T5", "A4500"],
    ["21", "20250522", "205239", 512, 256, 10, 8, 8, "GELU", "GELU", "scaled_dot", "ROPE", "A4500"],
    ["22", "20250522", "220634", 512, 256, 10, 8, 8, "GELU", "GELU", "AliBi", "AliBi", "A4500"],
], columns=[
    "runID","date", "time", "d_model", "seq_len", "n_layers", "n_heads", "n_output_layers",
    "FFNN_activation", "out_activation", "attention", "position_encoding", "GPU"
])


In [93]:
root_dir = os.getcwd()

In [94]:
# def load_single_summary(root_dir: str, subdir: str, df_lookup: pd.DataFrame = None) -> pd.DataFrame:
#     """
#     Load summary.csv from a single subdirectory and perform column consistency + merge with lookup.

#     Args:
#         root_dir (str): Root directory containing the subdirectory.
#         subdir (str): Subdirectory name (e.g., "20250518_152548").
#         df_lookup (pd.DataFrame, optional): Lookup table with config info.

#     Returns:
#         pd.DataFrame: Loaded and enriched summary dataframe.
#     """
#     folder_path = os.path.join(root_dir, subdir)
#     summary_path = os.path.join(folder_path, "summary.csv")

#     if not os.path.exists(summary_path):
#         print(f"[⚠️ Missing] {summary_path}")
#         return pd.DataFrame()

#     try:
#         df = pd.read_csv(summary_path)
#         if df.isna().any().any():
#             print(f"[NaN detected in CSV] {summary_path}")
#         df["folder"] = subdir

#         if "_" in subdir:
#             date, time_ = subdir.split("_")
#             df["date"] = date
#             df["time"] = time_

#         # Merge with lookup if given
#         if df_lookup is not None:
#             df_lookup = df_lookup.copy()
#             df_lookup["time"] = df_lookup["time"].astype(str)
#             df_lookup["date"] = df_lookup["date"].astype(str)
#             df["time"] = df["time"].astype(str)
#             df["date"] = df["date"].astype(str)
#             df = df.merge(df_lookup, on=["date", "time"], how="left")

#         # Final NaN check
#         if df.isna().any().any():
#             missing_cols = df.columns[df.isna().any()].tolist()
#             print(f"[NaN detected after merge] in columns: {missing_cols}")

#         return df

#     except Exception as e:
#         print(f"[ERROR] Failed to read {summary_path}: {e}")
#         return pd.DataFrame()


In [95]:
def load_single_summary(root_dir: str, subdir: str, df_lookup: pd.DataFrame = None) -> pd.DataFrame:
    """
    Load summary.csv from a single subdirectory and merge it with a lookup DataFrame on runID.

    Args:
        root_dir (str): Root directory containing the subdirectory.
        subdir (str): Subdirectory name (e.g., "7").
        df_lookup (pd.DataFrame, optional): Lookup table with config info.

    Returns:
        pd.DataFrame: Loaded and enriched summary dataframe.
    """
    folder_path = os.path.join(root_dir, subdir)
    summary_path = os.path.join(folder_path, "summary.csv")

    if not os.path.exists(summary_path):
        print(f"[⚠️ Missing] {summary_path}")
        return pd.DataFrame()

    try:
        df = pd.read_csv(summary_path)
        if df.isna().any().any():
            print(f"[NaN detected in CSV] {summary_path}")
        df["folder"] = subdir

        # Merge with lookup using runID
        if df_lookup is not None:
            df_lookup = df_lookup.copy()
            df_lookup["runID"] = df_lookup["runID"].astype(str)
            df["runID"] = df["runID"].astype(str)
            df = df.merge(df_lookup, on="runID", how="left")

        # Final NaN check
        if df.isna().any().any():
            missing_cols = df.columns[df.isna().any()].tolist()
            print(f"[NaN detected after merge] in columns: {missing_cols}")

        return df

    except Exception as e:
        print(f"[ERROR] Failed to read {summary_path}: {e}")
        return pd.DataFrame()

In [96]:
man =load_single_summary(root_dir, "1", df_lookup)

In [97]:
man

,runID,epoch,num_true_nu_e,AUC_true_nu_e_vs_rest,frac_pred_nu_e_on_true_nu_e_prob_gt_0.9,frac_pred_nu_e_on_true_nu_e_prob_gt_0.8,frac_pred_nu_e_on_true_nu_e_prob_lt_0.2,frac_pred_nu_e_on_true_nu_e_prob_lt_0.1,median_pred_nu_e_on_true_nu_e_prob,frac_pred_nu_mu_on_true_nu_e_prob_gt_0.9,...,d_model,seq_len,n_layers,n_heads,n_output_layers,FFNN_activation,out_activation,attention,position_encoding,GPU
0,1,49,10000,0.786701,0.9746,0.9756,0.0199,0.0189,1.000000,0.3102,...,512,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute,A4500
1,1,12,10000,0.939759,0.7338,0.8398,0.0184,0.0117,0.999060,0.0113,...,512,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute,A4500
2,1,20,10000,0.929177,0.8381,0.8848,0.0256,0.0186,1.000000,0.0209,...,512,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute,A4500
3,1,28,10000,0.906031,0.8853,0.9004,0.0374,0.0310,1.000000,0.0382,...,512,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute,A4500
4,1,24,10000,0.920121,0.8813,0.9069,0.0263,0.0196,1.000000,0.0253,...,512,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute,A4500
5,1,11,10000,0.937437,0.6744,0.7863,0.0301,0.0226,0.984831,0.0067,...,512,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute,A4500
6,1,32,10000,0.869685,0.9616,0.9643,0.0175,0.0148,1.000000,0.1138,...,512,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute,A4500


In [98]:
def load_all_summaries(root_dir: str, df_lookup: pd.DataFrame = None, verbose: bool = True) -> pd.DataFrame:
    """
    Loads all summary.csv files from subdirectories of root_dir and merges with training config info.
    Logs column coverage and detects which subdirs are missing expected columns.

    Args:
        root_dir (str): Root directory containing subdirectories like "7", "8", etc.
        df_lookup (pd.DataFrame, optional): DataFrame containing training config, keyed by 'runID'.
        verbose (bool): Whether to print detailed logs.

    Returns:
        pd.DataFrame: Concatenated summary dataframe with lookup info.
    """
    all_dfs = []
    column_tracker = {}

    for subdir in sorted(os.listdir(root_dir)):
        folder_path = os.path.join(root_dir, subdir)
        if not os.path.isdir(folder_path):
            continue

        df = load_single_summary(root_dir, subdir, df_lookup)
        if not df.empty:
            column_tracker[subdir] = set(df.columns)
            all_dfs.append(df)

    if not all_dfs:
        if verbose:
            print("[INFO] No valid summary files found.")
        return pd.DataFrame()

    full_col_set = set.union(*column_tracker.values())

    if verbose:
        print("\n🔍 Column consistency check:")
        for subdir, cols in column_tracker.items():
            missing = full_col_set - cols
            if missing:
                print(f"[❌] {subdir} is missing: {sorted(missing)}")
            else:
                print(f"[✅] {subdir} has all columns.")

    for df in all_dfs:
        missing_cols = full_col_set - set(df.columns)
        for col in missing_cols:
            df[col] = pd.NA

    df_all = pd.concat(all_dfs, ignore_index=True)

    # Optional: sort if runID and epoch exist
    if "runID" in df_all.columns:
        df_all["runID"] = df_all["runID"].astype(str)
    if "epoch" in df_all.columns:
        df_all["epoch"] = pd.to_numeric(df_all["epoch"], errors='coerce')
        df_all.sort_values(by=["runID", "epoch"], inplace=True)

    if verbose and df_all.isna().any().any():
        missing_cols = df_all.columns[df_all.isna().any()].tolist()
        print(f"\n[❗] NaN values detected in merged dataframe columns: {missing_cols}")

    return df_all


In [80]:
summaries = load_all_summaries(root_dir, df_lookup)

[⚠️ Missing] /lustre/hpc/icecube/cyan/factory/IceCubeTransformer/Plot_inference/20250519_113232/summary.csv

🔍 Column consistency check:
[✅] 20250518_152548 has all columns.
[✅] 20250518_153151 has all columns.
[✅] 20250518_172719 has all columns.
[✅] 20250518_172825 has all columns.
[✅] 20250518_174841 has all columns.
[✅] 20250518_174943 has all columns.
[✅] 20250518_233656 has all columns.
[✅] 20250518_233931 has all columns.
[✅] 20250518_235356 has all columns.
[✅] 20250518_235535 has all columns.
[✅] 20250519_111436 has all columns.
[✅] 20250519_111753 has all columns.
[✅] 20250519_112057 has all columns.
[✅] 20250519_173628 has all columns.
[✅] 20250519_180656 has all columns.
[✅] 20250521_121428 has all columns.
[✅] 20250521_121701 has all columns.
[✅] 20250521_122145 has all columns.


In [81]:
# columns with nan
print(summaries.columns[summaries.isna().any()].tolist())
print(summaries[summaries.isna().any(axis=1)])

[]
Empty DataFrame
Columns: [checkpoint, num_true_nu_e, AUC_true_nu_e_vs_rest, frac_pred_nu_e_on_true_nu_e_prob_gt_0.9, frac_pred_nu_e_on_true_nu_e_prob_gt_0.8, frac_pred_nu_e_on_true_nu_e_prob_lt_0.2, frac_pred_nu_e_on_true_nu_e_prob_lt_0.1, median_pred_nu_e_on_true_nu_e_prob, frac_pred_nu_mu_on_true_nu_e_prob_gt_0.9, frac_pred_nu_mu_on_true_nu_e_prob_gt_0.8, median_pred_nu_mu_on_true_nu_e_prob, frac_pred_nu_tau_on_true_nu_e_prob_gt_0.9, frac_pred_nu_tau_on_true_nu_e_prob_gt_0.8, median_pred_nu_tau_on_true_nu_e_prob, num_true_nu_mu, AUC_true_nu_mu_vs_rest, frac_pred_nu_mu_on_true_nu_mu_prob_gt_0.9, frac_pred_nu_mu_on_true_nu_mu_prob_gt_0.8, frac_pred_nu_mu_on_true_nu_mu_prob_lt_0.2, frac_pred_nu_mu_on_true_nu_mu_prob_lt_0.1, median_pred_nu_mu_on_true_nu_mu_prob, frac_pred_nu_e_on_true_nu_mu_prob_gt_0.9, frac_pred_nu_e_on_true_nu_mu_prob_gt_0.8, median_pred_nu_e_on_true_nu_mu_prob, frac_pred_nu_tau_on_true_nu_mu_prob_gt_0.9, frac_pred_nu_tau_on_true_nu_mu_prob_gt_0.8, median_pred_nu_ta

In [82]:
def show_top10_tau_auc(summaries):
    """
    Displays the top 10 rows with the highest AUC values from the summaries DataFrame.
    """
    top_10_auc = summaries.nlargest(12, 'AUC_true_nu_tau_vs_rest')
    return top_10_auc[['AUC_true_nu_tau_vs_rest','id', 'checkpoint','d_model', 'seq_len', 'n_layers', 'n_heads', 'n_output_layers', 'FFNN_activation', 'out_activation', 'attention', 'position_encoding']].sort_values(by='AUC_true_nu_tau_vs_rest', ascending=False).reset_index(drop=True)
show_top10_tau_auc(summaries)

,AUC_true_nu_tau_vs_rest,id,checkpoint,d_model,seq_len,n_layers,n_heads,n_output_layers,FFNN_activation,out_activation,attention,position_encoding
0,0.814296,19,23-mid,512,256,10,8,8,GELU,SiLU,T5,T5
1,0.814296,19,23-val_loss=0.111,512,256,10,8,8,GELU,SiLU,T5,T5
2,0.813478,19,20-mid,512,256,10,8,8,GELU,SiLU,T5,T5
3,0.812091,19,21-val_loss=0.110,512,256,10,8,8,GELU,SiLU,T5,T5
4,0.812059,19,26-mid,512,256,10,8,8,GELU,SiLU,T5,T5
5,0.809326,19,19-val_loss=0.110,512,256,10,8,8,GELU,SiLU,T5,T5
6,0.808617,18,23-mid,512,256,10,8,8,GELU,SiLU,AliBi,AliBi
7,0.808487,19,17-mid,512,256,10,8,8,GELU,SiLU,T5,T5
8,0.808487,19,17-val_loss=0.109,512,256,10,8,8,GELU,SiLU,T5,T5
9,0.807440,18,21-val_loss=0.111,512,256,10,8,8,GELU,SiLU,AliBi,AliBi


In [ ]:
def add_

# Best Composite Fraction

In [83]:
def show_top10_tau_composite_fraction_090(summaries):
    """
    Displays the top 10 rows based on the tau fraction:
    (frac_tau_prob_gt_0.9 - frac_nu_e_tau_prob_gt_0.9 - frac_nu_mu_tau_prob_gt_0.9)
    """
    summaries = summaries.copy()
    summaries["tau_composite_fraction090"] = (
        summaries["frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.9"]
        - summaries["frac_pred_nu_tau_on_true_nu_mu_prob_gt_0.9"]
        - summaries["frac_pred_nu_tau_on_true_nu_e_prob_gt_0.9"]
        - summaries["frac_pred_nu_tau_on_true_nu_tau_prob_lt_0.1"]
    )
    top_10 = summaries.nlargest(11, "tau_composite_fraction090")
    return top_10[['tau_composite_fraction090','id', 'checkpoint','d_model', 'seq_len', 'n_layers', 'n_heads', 'n_output_layers', 'FFNN_activation', 'out_activation', 'attention', 'position_encoding']].sort_values(by='tau_composite_fraction090', ascending=False).reset_index(drop=True)
show_top10_tau_composite_fraction_090(summaries)


,tau_composite_fraction090,id,checkpoint,d_model,seq_len,n_layers,n_heads,n_output_layers,FFNN_activation,out_activation,attention,position_encoding
0,0.3351,15,20-mid,512,256,10,8,8,SiLU,SiLU,T5,T5
1,0.3074,9,29-mid,512,256,10,8,8,ReLu,ReLu,scaled_dot,Absolute
2,0.3027,16,32-mid,512,256,10,8,8,SiLU,SiLU,scaled_dot,ROPE
3,0.3010,15,23-mid,512,256,10,8,8,SiLU,SiLU,T5,T5
4,0.2845,16,29-mid,512,256,10,8,8,SiLU,SiLU,scaled_dot,ROPE
5,0.2795,15,26-mid,512,256,10,8,8,SiLU,SiLU,T5,T5
6,0.2744,9,20-mid,512,256,10,8,8,ReLu,ReLu,scaled_dot,Absolute
7,0.2721,9,20-val_loss=0.124,512,256,10,8,8,ReLu,ReLu,scaled_dot,Absolute
8,0.2719,15,29-mid,512,256,10,8,8,SiLU,SiLU,T5,T5
9,0.2687,19,26-mid,512,256,10,8,8,GELU,SiLU,T5,T5


In [84]:
def show_top10_tau_composite_fraction_080(summaries):
    """
    Displays the top 10 rows based on the tau fraction:
    (frac_tau_prob_gt_0.8 - frac_nu_e_tau_prob_gt_0.8 - frac_nu_mu_tau_prob_gt_0.8)
    """
    summaries = summaries.copy()
    summaries["tau_composite_fraction080"] = (
        summaries["frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.8"]
        - summaries["frac_pred_nu_tau_on_true_nu_mu_prob_gt_0.8"]
        - summaries["frac_pred_nu_tau_on_true_nu_e_prob_gt_0.8"]
        - summaries["frac_pred_nu_tau_on_true_nu_tau_prob_lt_0.2"]
    )
    top_10 = summaries.nlargest(11, "tau_composite_fraction080")
    return top_10[['tau_composite_fraction080','id', 'checkpoint','d_model', 'seq_len', 'n_layers', 'n_heads', 'n_output_layers','FFNN_activation', 'out_activation', 'attention', 'position_encoding']].sort_values(by='tau_composite_fraction080', ascending=False).reset_index(drop=True)
show_top10_tau_composite_fraction_080(summaries)

,tau_composite_fraction080,id,checkpoint,d_model,seq_len,n_layers,n_heads,n_output_layers,FFNN_activation,out_activation,attention,position_encoding
0,0.1892,15,20-mid,512,256,10,8,8,SiLU,SiLU,T5,T5
1,0.1864,15,26-mid,512,256,10,8,8,SiLU,SiLU,T5,T5
2,0.1802,15,29-mid,512,256,10,8,8,SiLU,SiLU,T5,T5
3,0.1797,15,23-mid,512,256,10,8,8,SiLU,SiLU,T5,T5
4,0.1765,19,26-mid,512,256,10,8,8,GELU,SiLU,T5,T5
5,0.1719,19,32-mid,512,256,10,8,8,GELU,SiLU,T5,T5
6,0.1660,17,26-mid,512,256,10,8,8,GELU,SiLU,scaled_dot,ROPE
7,0.1542,15,last,512,256,10,8,8,SiLU,SiLU,T5,T5
8,0.1534,19,35-mid,512,256,10,8,8,GELU,SiLU,T5,T5
9,0.1413,15,35-mid,512,256,10,8,8,SiLU,SiLU,T5,T5


# Pure High Confidence 

In [85]:
def show_top10_tau_090(summaries):
    top_10 = summaries.nlargest(10, 'frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.9')
    return top_10[['frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.9','id', 'checkpoint','d_model', 'seq_len', 'n_layers', 'n_heads', 'n_output_layers', 'out_activation', 'attention', 'position_encoding']].sort_values(by='frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.9', ascending=False).reset_index(drop=True)
show_top10_tau_090(summaries)

,frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.9,id,checkpoint,d_model,seq_len,n_layers,n_heads,n_output_layers,out_activation,attention,position_encoding
0,0.5881,15,last,512,256,10,8,8,SiLU,T5,T5
1,0.5854,9,last,512,256,10,8,8,ReLu,scaled_dot,Absolute
2,0.5742,19,last,512,256,10,8,8,SiLU,T5,T5
3,0.5676,16,last,512,256,10,8,8,SiLU,scaled_dot,ROPE
4,0.5659,9,32-mid,512,256,10,8,8,ReLu,scaled_dot,Absolute
5,0.5442,17,last,512,256,10,8,8,SiLU,scaled_dot,ROPE
6,0.5428,16,32-mid,512,256,10,8,8,SiLU,scaled_dot,ROPE
7,0.5416,7,last,512,256,10,8,2,ReLu,scaled_dot,Absolute
8,0.5360,9,35-mid,512,256,10,8,8,ReLu,scaled_dot,Absolute
9,0.5336,9,29-mid,512,256,10,8,8,ReLu,scaled_dot,Absolute


In [86]:
def show_top10_tau_080(summaries):
    top_10 = summaries.nlargest(10, 'frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.8')
    return top_10[['frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.8','id', 'checkpoint','d_model', 'seq_len', 'n_layers', 'n_heads', 'n_output_layers','FFNN_activation', 'out_activation', 'attention', 'position_encoding']].sort_values(by='frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.8', ascending=False).reset_index(drop=True)
show_top10_tau_080(summaries)

,frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.8,id,checkpoint,d_model,seq_len,n_layers,n_heads,n_output_layers,FFNN_activation,out_activation,attention,position_encoding
0,0.6203,15,last,512,256,10,8,8,SiLU,SiLU,T5,T5
1,0.6078,19,last,512,256,10,8,8,GELU,SiLU,T5,T5
2,0.5946,9,last,512,256,10,8,8,ReLu,ReLu,scaled_dot,Absolute
3,0.5853,9,32-mid,512,256,10,8,8,ReLu,ReLu,scaled_dot,Absolute
4,0.5834,16,last,512,256,10,8,8,SiLU,SiLU,scaled_dot,ROPE
5,0.5741,16,32-mid,512,256,10,8,8,SiLU,SiLU,scaled_dot,ROPE
6,0.5665,17,last,512,256,10,8,8,GELU,SiLU,scaled_dot,ROPE
7,0.5627,7,last,512,256,10,8,2,ReLu,ReLu,scaled_dot,Absolute
8,0.5593,9,29-mid,512,256,10,8,8,ReLu,ReLu,scaled_dot,Absolute
9,0.5538,19,35-mid,512,256,10,8,8,GELU,SiLU,T5,T5


# Worst Composite fraction

In [87]:
def show_bottom10_tau_auc(summaries):
    bottom_10_auc = summaries.nsmallest(10, 'AUC_true_nu_tau_vs_rest')
    return bottom_10_auc[['AUC_true_nu_tau_vs_rest','id', 'checkpoint','d_model', 'seq_len', 'n_layers', 'n_heads', 'n_output_layers','FFNN_activation', 'out_activation', 'attention', 'position_encoding']].sort_values(by='AUC_true_nu_tau_vs_rest', ascending=True).reset_index(drop=True)
show_bottom10_tau_auc(summaries)
    

,AUC_true_nu_tau_vs_rest,id,checkpoint,d_model,seq_len,n_layers,n_heads,n_output_layers,FFNN_activation,out_activation,attention,position_encoding
0,0.514626,2,last,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
1,0.522558,2,32-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
2,0.530328,2,28-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
3,0.545704,2,24-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
4,0.550010,5,last,512,256,6,1,2,ReLu,ReLu,scaled_dot,Absolute
5,0.563507,5,35-mid,512,256,6,1,2,ReLu,ReLu,scaled_dot,Absolute
6,0.569385,2,20-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
7,0.581819,5,32-mid,512,256,6,1,2,ReLu,ReLu,scaled_dot,Absolute
8,0.591785,1,last,512,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
9,0.606860,1,32-mid,512,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute


In [88]:
def show_bottom10_tau_composite_fraction_090(summaries):
    """
    Displays the bottom 10 rows based on the tau fraction:
    (frac_tau_prob_gt_0.9 - frac_nu_e_tau_prob_gt_0.9 - frac_nu_mu_tau_prob_gt_0.9)
    """
    summaries = summaries.copy()
    summaries["tau_composite_fraction090"] = (
        summaries["frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.9"]
        - summaries["frac_pred_nu_tau_on_true_nu_mu_prob_gt_0.9"]
        - summaries["frac_pred_nu_tau_on_true_nu_e_prob_gt_0.9"]
        - summaries["frac_pred_nu_tau_on_true_nu_tau_prob_lt_0.1"]
    )
    bottom_10 = summaries.nsmallest(10, "tau_composite_fraction090")
    return bottom_10[['tau_composite_fraction090','id', 'checkpoint','d_model', 'seq_len', 'n_layers', 'n_heads', 'n_output_layers','FFNN_activation', 'out_activation', 'attention', 'position_encoding']].sort_values(by='tau_composite_fraction090', ascending=True).reset_index(drop=True)
show_bottom10_tau_composite_fraction_090(summaries)

,tau_composite_fraction090,id,checkpoint,d_model,seq_len,n_layers,n_heads,n_output_layers,FFNN_activation,out_activation,attention,position_encoding
0,-1.7402,2,last,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
1,-1.6848,2,32-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
2,-1.6716,2,28-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
3,-1.5242,2,24-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
4,-1.4281,5,35-mid,512,256,6,1,2,ReLu,ReLu,scaled_dot,Absolute
5,-1.4060,5,32-mid,512,256,6,1,2,ReLu,ReLu,scaled_dot,Absolute
6,-1.3215,2,20-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
7,-1.2367,5,last,512,256,6,1,2,ReLu,ReLu,scaled_dot,Absolute
8,-1.1968,1,last,512,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
9,-1.1642,3,35-mid,512,256,6,4,2,ReLu,ReLu,scaled_dot,Absolute


In [89]:
def show_bottom10_tau_composite_fraction_080(summaries):
    """
    Displays the bottom 10 rows based on the tau fraction:
    (frac_tau_prob_gt_0.8 - frac_nu_e_tau_prob_gt_0.8 - frac_nu_mu_tau_prob_gt_0.8)
    """
    summaries = summaries.copy()
    summaries["tau_composite_fraction080"] = (
        summaries["frac_pred_nu_tau_on_true_nu_tau_prob_gt_0.8"]
        - summaries["frac_pred_nu_tau_on_true_nu_mu_prob_gt_0.8"]
        - summaries["frac_pred_nu_tau_on_true_nu_e_prob_gt_0.8"]
        - summaries["frac_pred_nu_tau_on_true_nu_tau_prob_lt_0.2"]
    )
    bottom_10 = summaries.nsmallest(10, "tau_composite_fraction080")
    return bottom_10[['tau_composite_fraction080','id', 'checkpoint','d_model', 'seq_len', 'n_layers', 'n_heads', 'n_output_layers','FFNN_activation', 'out_activation', 'attention', 'position_encoding']].sort_values(by='tau_composite_fraction080', ascending=True).reset_index(drop=True)
show_bottom10_tau_composite_fraction_080(summaries)

,tau_composite_fraction080,id,checkpoint,d_model,seq_len,n_layers,n_heads,n_output_layers,FFNN_activation,out_activation,attention,position_encoding
0,-1.7495,2,last,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
1,-1.6968,2,32-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
2,-1.6865,2,28-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
3,-1.5498,2,24-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
4,-1.4713,5,35-mid,512,256,6,1,2,ReLu,ReLu,scaled_dot,Absolute
5,-1.4226,5,32-mid,512,256,6,1,2,ReLu,ReLu,scaled_dot,Absolute
6,-1.3727,2,20-mid,256,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
7,-1.3251,5,last,512,256,6,1,2,ReLu,ReLu,scaled_dot,Absolute
8,-1.2071,1,last,512,128,6,4,2,ReLu,ReLu,scaled_dot,Absolute
9,-1.1898,3,35-mid,512,256,6,4,2,ReLu,ReLu,scaled_dot,Absolute
